In [25]:

import sys
!{sys.executable} -m pip install pandas numpy duckdb

import duckdb
import numpy as np
import pandas as pd
from pathlib import Path
import os

# 1. Encontrar o diretório base do projeto
try:
    BASE_DIR = Path(__file__).resolve().parent.parent  # Para scripts .py
except NameError:
    BASE_DIR = Path(os.getcwd()).parent  # Para notebooks Jupyter

# 2. Construir caminhos absolutos
PARQUET_DIR = BASE_DIR / "dados" / "refined"
paths = {
    "fato_vendas": PARQUET_DIR / "fato_vendas.parquet",
    "dim_cliente": PARQUET_DIR / "dim_cliente.parquet",
    "dim_tempo": PARQUET_DIR / "dim_tempo.parquet",
    "dim_produto": PARQUET_DIR / "dim_produto.parquet"
}

# 3. Verificar se os arquivos existem
for nome, caminho in paths.items():
    if not caminho.exists():
        print(f"ERRO: Arquivo não encontrado - {caminho}")
        print("Execute esta célula para ver o diretório atual:")
        print("import os; print(os.getcwd())")
        raise FileNotFoundError(f"Arquivo {nome} não encontrado")

# 4. Conexão DuckDB e criação das views
conn = duckdb.connect()

# Criar views com caminhos absolutos garantidos
for nome, caminho in paths.items():
        conn.execute(f"""
        CREATE OR REPLACE TEMP VIEW {nome} AS 
        SELECT * FROM '{caminho}'
        """)


### 1. Análise de Vendas por Categoria e Período (dim_produto + dim_tempo)

Insight: Identificar sazonalidade por categoria e dias da semana com maior volume.

In [26]:
resultado = conn.execute("""
SELECT 
    p.cat_categoria,
    t.nu_mes,
    t.no_dia_semana,
    SUM(f.vr_total) AS receita_total,
    COUNT(f.id_compra) AS volume_vendas,
    SUM(f.vr_frete) AS total_frete
FROM 'fato_vendas' f
JOIN 'dim_tempo' t ON f.id_tempo = t.id_tempo
JOIN 'dim_produto' p ON f.id_produto = p.id_produto
GROUP BY p.cat_categoria, t.nu_mes, t.no_dia_semana
ORDER BY p.cat_categoria, t.nu_mes;
                         
""").fetchdf()
display(resultado)

,cat_categoria,nu_mes,no_dia_semana,receita_total,volume_vendas,total_frete
0,Alimentos e bebidas,1,Wednesday,1895.120022,8,71.620000
1,Alimentos e bebidas,1,Tuesday,2416.080030,13,68.779999
2,Alimentos e bebidas,1,Thursday,1914.920021,10,90.920002
3,Alimentos e bebidas,1,Friday,1818.190056,12,72.390001
4,Alimentos e bebidas,1,Saturday,2205.250080,12,104.050000
...,...,...,...,...,...,...
561,Papelaria e escritório,12,Tuesday,1237.949995,15,29.250000
562,Papelaria e escritório,12,Thursday,1069.650016,13,46.250000
563,Papelaria e escritório,12,Saturday,2619.040043,24,124.140000
564,Papelaria e escritório,12,Sunday,2195.330010,22,55.330000


### 2. Ticket Médio por Faixa Etária e Método de Pagamento (dim_cliente + dim_produto)

Insight: Descobrir quais perfis de cliente geram mais receita por método de pagamento.

In [27]:
resultado = conn.execute("""
SELECT 
    c.cat_faixa_etaria,
    p.cat_meio_pagamento,
    AVG(f.vr_total) AS ticket_medio,
    SUM(f.vr_total) AS receita_total,
    COUNT(*) AS transacoes
FROM 'fato_vendas' f
JOIN 'dim_cliente' c ON f.sk_cliente = c.sk_cliente
JOIN 'dim_produto' p ON f.id_produto = p.id_produto
GROUP BY c.cat_faixa_etaria, p.cat_meio_pagamento
ORDER BY receita_total DESC;
""").fetchdf()
display(resultado)

,cat_faixa_etaria,cat_meio_pagamento,ticket_medio,receita_total,transacoes
0,35-44,PIX,2849.069978,3.185260e+06,1118
1,35-44,Cartão de Crédito,2667.296986,3.072726e+06,1152
2,45-54,Cartão de Crédito,2810.008946,1.969816e+06,701
3,25-34,Cartão de Crédito,2724.347370,1.882524e+06,691
4,45-54,PIX,2732.000870,1.743017e+06,638
5,25-34,PIX,2787.531391,1.039749e+06,373
6,55-64,Boleto,3286.277693,7.196948e+05,219
7,18-24,Cartão de Crédito,2862.620468,6.383644e+05,223
8,25-34,Cartão de Débito,1891.127640,5.635560e+05,298
9,25-34,Boleto,3104.927230,4.781588e+05,154


### 3. Crescimento Anual por Região e Categoria (dim_tempo + dim_cliente + dim_produto)

Insight: Identificar categorias em crescimento por região para investimento estratégico.

In [28]:
resultado = conn.execute("""
WITH vendas_anuais AS (
    SELECT
        t.nu_ano,
        c.cat_regiao,
        p.cat_categoria,
        SUM(f.vr_total) AS receita
    FROM 'fato_vendas' f
    JOIN 'dim_tempo' t ON f.id_tempo = t.id_tempo
    JOIN 'dim_cliente' c ON f.sk_cliente = c.sk_cliente
    JOIN 'dim_produto' p ON f.id_produto = p.id_produto
    GROUP BY t.nu_ano, c.cat_regiao, p.cat_categoria
)
SELECT 
    *,
    (receita - LAG(receita) OVER (PARTITION BY cat_regiao, cat_categoria ORDER BY nu_ano)) AS crescimento
FROM vendas_anuais
ORDER BY cat_regiao, cat_categoria, nu_ano;
                         """).fetchdf()
display(resultado)

,nu_ano,cat_regiao,cat_categoria,receita,crescimento
0,2023,Centro-Oeste,Alimentos e bebidas,1.769668e+04,NaN
1,2023,Centro-Oeste,Beleza e cuidados pessoais,3.273788e+04,NaN
2,2023,Centro-Oeste,Casa e limpeza,3.003170e+04,NaN
3,2023,Centro-Oeste,Eletrodomésticos,4.712125e+05,NaN
4,2023,Centro-Oeste,Eletrônicos,8.296938e+05,NaN
5,2023,Centro-Oeste,Ferramentas e construção,2.130026e+04,NaN
6,2023,Centro-Oeste,Livros,5.483410e+03,NaN
7,2023,Centro-Oeste,Papelaria e escritório,3.509990e+03,NaN
8,2023,Nordeste,Alimentos e bebidas,7.989140e+04,NaN
9,2023,Nordeste,Beleza e cuidados pessoais,1.370673e+05,NaN


### 4. Eficiência de Frete por Categoria e Estado (dim_produto + dim_cliente)

Insight: Otimizar política de fretes para combinações categoria/estado menos eficientes.

In [29]:
resultado = conn.execute("""
SELECT 
    p.cat_categoria,
    c.sg_uf,
    SUM(f.vr_frete) / SUM(f.vr_total) AS frete_sobre_venda,
    AVG(f.vr_total) AS ticket_medio,
    COUNT(*) AS volume
FROM 'fato_vendas' f
JOIN 'dim_produto' p ON f.id_produto = p.id_produto
JOIN 'dim_cliente' c ON f.sk_cliente = c.sk_cliente
GROUP BY p.cat_categoria, c.sg_uf
HAVING COUNT(*) > 30  -- Filtra estados com volume significativo
ORDER BY frete_sobre_venda DESC;

""").fetchdf()
display(resultado)

,cat_categoria,sg_uf,frete_sobre_venda,ticket_medio,volume
0,Casa e limpeza,Rio Grande do Norte,0.074557,374.093803,42
1,Casa e limpeza,Pernambuco,0.072299,343.358803,67
2,Alimentos e bebidas,Bahia,0.071362,180.091506,113
3,Eletrodomésticos,Bahia,0.070516,6247.724579,52
4,Beleza e cuidados pessoais,Paraíba,0.070491,467.995500,49
...,...,...,...,...,...
58,Alimentos e bebidas,São Paulo,0.000685,153.932365,364
59,Ferramentas e construção,São Paulo,0.000649,500.207653,107
60,Beleza e cuidados pessoais,São Paulo,0.000570,389.189474,289
61,Casa e limpeza,São Paulo,0.000382,299.284099,256


### 5. Análise de Satisfação por Método de Pagamento e Dia da Semana (dim_produto + dim_tempo)

Insight: Identificar combinações pagamento/dia com maior satisfação para replicar estratégias.

In [30]:
resultado = conn.execute("""

SELECT 
    p.cat_meio_pagamento,
    t.no_dia_semana,
    AVG(f.nu_avaliacao) AS avaliacao_media,
    COUNT(*) AS volume_vendas,
    SUM(f.vr_cashback) AS total_cashback
FROM 'fato_vendas' f
JOIN 'dim_produto' p ON f.id_produto = p.id_produto
JOIN 'dim_tempo' t ON f.id_tempo = t.id_tempo
GROUP BY p.cat_meio_pagamento, t.no_dia_semana
ORDER BY avaliacao_media DESC;
                         
""").fetchdf()
display(resultado)

,cat_meio_pagamento,no_dia_semana,avaliacao_media,volume_vendas,total_cashback
0,Boleto,Tuesday,8.741573,89,23.0
1,Cartão de Débito,Sunday,8.727273,88,53.0
2,Cartão de Débito,Wednesday,8.644231,104,65.0
3,PIX,Tuesday,8.519553,358,148.0
4,Boleto,Sunday,8.500000,84,26.0
5,Cartão de Crédito,Friday,8.497899,476,201.0
6,Cartão de Crédito,Saturday,8.495220,523,224.0
7,PIX,Thursday,8.490358,363,123.0
8,Cartão de Débito,Friday,8.481013,79,48.0
9,PIX,Wednesday,8.475543,368,152.0


### 6. Clientes Premium (Cross-Dimension)

Insight: Perfil completo dos clientes de alto valor para programas de fidelidade.

In [31]:
resultado = conn.execute("""
WITH 
    clientes_premium AS (
        SELECT
            f.sk_cliente,
            SUM(f.vr_total) AS gasto_total,
            COUNT(DISTINCT f.id_produto) AS categorias_compradas
        FROM 'fato_vendas' f
        GROUP BY f.sk_cliente
        HAVING SUM(f.vr_total) > 10000  -- Defina seu critério de premium
    )
                            
SELECT 
    c.cat_faixa_etaria,
    c.cat_genero,
    c.cat_regiao,
    p.cat_meio_pagamento,
    AVG(cp.gasto_total) AS avg_gasto,
    COUNT(*) AS qtd_clientes
FROM clientes_premium cp
JOIN 'dim_cliente' c ON cp.sk_cliente = c.sk_cliente
JOIN 'fato_vendas' f ON cp.sk_cliente = f.sk_cliente
JOIN 'dim_produto' p ON f.id_produto = p.id_produto
GROUP BY c.cat_faixa_etaria, c.cat_genero, c.cat_regiao, p.cat_meio_pagamento;

""").fetchdf()
display(resultado)

,cat_faixa_etaria,cat_genero,cat_regiao,cat_meio_pagamento,avg_gasto,qtd_clientes
0,25-34,Feminino,Sudeste,Cartão de Débito,25673.503341,6
1,45-54,Masculino,Sudeste,Cartão de Crédito,19510.905166,29
2,45-54,Feminino,Sudeste,Cartão de Crédito,20793.706140,28
3,45-54,Feminino,Centro-Oeste,Cartão de Crédito,18630.403727,7
4,25-34,Masculino,Sudeste,Boleto,20653.422592,4
...,...,...,...,...,...,...
141,45-54,Feminino,Centro-Oeste,PIX,21711.250354,10
142,45-54,Feminino,Norte,PIX,18695.819336,2
143,35-44,Masculino,Sul,Boleto,31673.825963,3
144,55-64,Feminino,Nordeste,PIX,47945.388672,1


### 7. Análise de Satisfação

Insights: Identificar pontos de insatisfação por perfil de cliente.

In [32]:
resultado = conn.execute("""
SELECT 
    p.cat_meio_pagamento,
    c.cat_faixa_etaria,
    AVG(f.nu_avaliacao) AS avaliacao_media,
    COUNT(f.id_compra) AS volume
FROM fato_vendas f
JOIN dim_produto p ON f.id_produto = p.id_produto
JOIN dim_cliente c ON f.sk_cliente = c.sk_cliente
GROUP BY p.cat_meio_pagamento, c.cat_faixa_etaria
ORDER BY avaliacao_media DESC;
""").fetchdf()
display(resultado)

,cat_meio_pagamento,cat_faixa_etaria,avaliacao_media,volume
0,Cartão de Crédito,65+,9.500000,2
1,PIX,65+,9.250000,4
2,Boleto,45-54,8.850000,40
3,Cartão de Débito,55-64,8.777778,9
4,Cartão de Débito,45-54,8.666667,33
5,Cartão de Débito,18-24,8.651316,152
6,Cartão de Débito,35-44,8.581818,55
7,Boleto,18-24,8.562500,16
8,Boleto,55-64,8.552511,219
9,PIX,45-54,8.512539,638
